In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error, explained_variance_score
import seaborn as sns

In [2]:

df = pd.read_csv("diabetes.csv")

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# encoding all variables in the dataset
from sklearn.preprocessing import LabelEncoder

# Get the list of all object columns
object_variables = df.select_dtypes(include=['object']).columns.tolist()

# Find all the boolean columns
bool_variables = [col for col in object_variables if df[col].nunique() == 2]

# Find all the categorical columns with more than 2 unique values
cat_variables = [col for col in object_variables if df[col].nunique() > 2]

encoder = LabelEncoder()
df[bool_variables] = df[bool_variables].apply(encoder.fit_transform)

# get dummies for categorical variables
df = pd.get_dummies(df, columns=cat_variables, drop_first=True)

In [4]:
# Check for nans
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
# Split features and target
X = df.drop(columns=['DiabetesPedigreeFunction']).values
y = df['DiabetesPedigreeFunction'].values.reshape(-1, 1)

# Normalize features 
feature_scaler = StandardScaler()
X = feature_scaler.fit_transform(X)

# Normalize target variable
target_scaler = StandardScaler()
y = target_scaler.fit_transform(y)


# First, split the data into train and remaining (validation + test)
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.3, random_state=42)

# Second, split the remaining data into validation and test sets (e.g., 50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)


# Convert to PyTorch tensors
X_train, X_test, X_val = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32), torch.tensor(X_val, dtype=torch.float32)
y_train, y_test, y_val = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32) 

# create PyTorch dataloaders, YOU CAN ADJUST THE BATCH SIZES IF YOU WISH
# basic rule: bigger batch size => faster training, but more memory needed
# only shuffle the training dataset, NEVER SHUFFLE TEST/VAL
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

In [6]:
# all common hyperparameters we can finetune in the future

# sigma = standard deviation of the noise term (this is often optimized)
# if model is too confident and OVERFITS at the same time, INCREASE sigma
# if predictions are too noisy, too much variation or uncertain  => DECREASE sigma
# COMMON VALUES 0.1 - 0.5

# lambda_kl = amount of KL in the loss function => regularization parameter to avoid OVERFITTING
# if model overfits => increase, if model underfits => decrease
# COMMON VALUES 0.0+ - 0.2

# mu = mean of distribution (you can usually use the defaul 0 in classification)
# if model overfits => decrease mu
# if model underfits => increase mu
# mu = 0 => in the middle of standard distribution (range -3 to +3) => this needs be checked!
# COMMON VALUES: usually 0 in classification

# mc_samples_eval = monte carlo sample size used to approximate distributions in weights
# more samples used => more accurate uncertainty estimation (but heavier to compute)
# if uncertainty is not optimal, try increasing mc_sample -size a little bit
# if model training is too heavy or underfits, you can decrease the sample size
# COMMON VALUES: 25 - 200

mu = 0
sigma = 0.3
lambda_kl = 0.05
mc_samples_eval = 30

# remember to adjust the BNN network structure in the below cell if needed
# amount of layers, layer sizes, amount of dropout regularization etc.

# learning rate is same as in ANN => controls the aggressiveness of optimization
# too large value => model overshoots optimization attempts
# too small value => model learns too slowly
# COMMON VALUES similar to ANN learning rates, small values (0.03 or less), usually 0.1 or less
bnn_learning_rate = 0.01

# weight_decay = regularization applied to model weights (probability distributions)
# penalizes large weights IN ORDER to prevent OVERFITTING
# if model overfits => you can try increasing weight decay, if underfits => try decreasing
# COMMON VALUES are usually quite small, 0.0001 to 0.0005
bnn_weight_decay = 0.0003

In [7]:
# Bayesian Neural Network -model (BNN) with deeper layers and Monte-Carlo dropout
class BayesianNN(nn.Module):
    def __init__(self, input_dim, output_dim, mu=0, sigma=0.5):
        super().__init__()
        # DEFINE ALL LAYERS THAT WILL BE COMBINED INTO ONE MODEL IN forward()
        # NOTE: Torch requires as to "chain" connected layer => out_features and in_features have to match in connected layers
        # for example "in_features=96, out_features=48" means that previous layers to have out_features of 96
        # and the next layer has to have in features of 48
        self.fc1 = bnn.BayesLinear(prior_mu=mu, prior_sigma=sigma, in_features=input_dim, out_features=128)
        self.fc2 = bnn.BayesLinear(prior_mu=mu, prior_sigma=sigma, in_features=128, out_features=64)
        self.fc3 = bnn.BayesLinear(prior_mu=mu, prior_sigma=sigma, in_features=64, out_features=32)
        self.fc4 = bnn.BayesLinear(prior_mu=mu, prior_sigma=sigma, in_features=32, out_features=16)
        self.fc5 = bnn.BayesLinear(prior_mu=mu, prior_sigma=sigma, in_features=16, out_features=output_dim)
        self.relu = nn.ReLU()

        # adjust dropout rate to increase regularization
        self.dropout = nn.Dropout(0.3)  

        # THE BATCH NORMALIZATION SIZES HAVE THE MATCH THE OUTPUT OF THE CORRESPONDING LAYER
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(64)
        self.batch_norm3 = nn.BatchNorm1d(32)
        self.batch_norm4 = nn.BatchNorm1d(16)
    
    # "forward pass", if mc_dropout = True => adds a dropout layer between the BayesLinear -layers
    def forward(self, x, mc_dropout=False):
        # NOTE: batch normalization helps in stabilizing the training process
        x = self.batch_norm1(self.relu(self.fc1(x))) 
        x = self.dropout(x) if mc_dropout else x
        x = self.batch_norm2(self.relu(self.fc2(x)))
        x = self.dropout(x) if mc_dropout else x
        x = self.batch_norm3(self.relu(self.fc3(x)))
        x = self.dropout(x) if mc_dropout else x
        x = self.batch_norm4(self.relu(self.fc4(x)))
        x = self.dropout(x) if mc_dropout else x
        return self.fc5(x) 

In [8]:
# model, loss, optimizer
# NOTE! Regression, OUTPUT DIMENSION IS 1 VARIABLE!
model = BayesianNN(input_dim=X.shape[1], 
                   output_dim=1,
                   sigma=sigma,
                   mu=mu
                   )

# Loss function for REGRESSION
criterion = nn.MSELoss() 

# Bayesian KL Divergence Loss (efficient regularization for BNN)
bkl_loss = bnn.BKLLoss(reduction='mean')  

# AdamW is common in BNN
optimizer = optim.AdamW(model.parameters(), lr=bnn_learning_rate, weight_decay=bnn_weight_decay)

In [9]:
# Evaluate function with uncertainty estimation
def evaluate(model, loader, mc_samples=50):
    model.eval() # Set model to evaluation mode
    total_loss = 0
    uncertainties = []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            outputs_list = [model(X_batch, mc_dropout=True) for _ in range(mc_samples)]
            outputs_stack = torch.stack(outputs_list)  # Shape: (mc_samples, batch_size, num_classes)
            outputs_mean = outputs_stack.mean(dim=0)  # Average prediction
            outputs_std = outputs_stack.std(dim=0)  # Standard deviation (uncertainty)

            loss = criterion(outputs_mean, y_batch)
            total_loss += loss.item()


            uncertainties.extend(outputs_std.mean(dim=1).cpu().numpy())  # Store uncertainty per sample

    r2 = r2_score(y_batch.cpu().numpy(), outputs_mean.cpu().numpy())  # Use R^2 score for regression tasks
    avg_loss = total_loss / len(loader) # Average loss across all batches
    avg_uncertainty = np.mean(uncertainties)  # Average uncertainty across all samples

    print(f"Val. confidence score: {r2:.4f}, Val. loss: {avg_loss:.4f}, Avg. Uncertainty: {avg_uncertainty:.4f}")

    return r2, avg_loss, avg_uncertainty, uncertainties  # Return per-sample uncertainty if needed

# Training loop with BKLLoss
def train(model, loader, epochs=50):
    model.train()
    history = {}
    history['train_loss'] = []
    history['val_loss'] = []
    history['r2'] = []
    history['avg_uncertainty'] = []
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            outputs = model(X_batch, mc_dropout=True)
            target_loss = criterion(outputs, y_batch) # Compute target loss
            kl_loss = bkl_loss(model)  # Compute BKL loss
            loss = target_loss + lambda_kl * kl_loss  # Combine losses
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {total_loss/len(loader):.4f}")

        r2, val_loss, avg_uncertainty, _ = evaluate(model, val_loader)
        history['train_loss'].append(total_loss / len(loader))
        history['val_loss'].append(val_loss)
        history['r2'].append(r2)
        history['avg_uncertainty'].append(avg_uncertainty)

        # set model to training mode for next epoch
        model.train()
        
    return history

In [10]:
history = train(model, train_loader, epochs=4000) 

Epoch 1/4000, Training Loss: 3.7374
Val. confidence score: -0.1057, Val. loss: 0.9702, Avg. Uncertainty: 1.0094
Epoch 2/4000, Training Loss: 3.7411
Val. confidence score: 0.0540, Val. loss: 0.8752, Avg. Uncertainty: 0.7806
Epoch 3/4000, Training Loss: 2.8285
Val. confidence score: -0.0139, Val. loss: 0.9023, Avg. Uncertainty: 0.7104
Epoch 4/4000, Training Loss: 2.1463
Val. confidence score: -0.0248, Val. loss: 0.9327, Avg. Uncertainty: 0.6442
Epoch 5/4000, Training Loss: 1.7053
Val. confidence score: -0.0427, Val. loss: 0.9300, Avg. Uncertainty: 0.5988
Epoch 6/4000, Training Loss: 2.0000
Val. confidence score: -0.0451, Val. loss: 0.9216, Avg. Uncertainty: 0.5830
Epoch 7/4000, Training Loss: 1.7988
Val. confidence score: 0.0083, Val. loss: 0.9010, Avg. Uncertainty: 0.5284
Epoch 8/4000, Training Loss: 1.7676
Val. confidence score: -0.0418, Val. loss: 0.9154, Avg. Uncertainty: 0.5091
Epoch 9/4000, Training Loss: 1.7393
Val. confidence score: -0.0203, Val. loss: 0.9019, Avg. Uncertainty: 0

In [11]:
df.iloc[542] 

Pregnancies                 10.000
Glucose                     90.000
BloodPressure               85.000
SkinThickness               32.000
Insulin                      0.000
BMI                         34.900
DiabetesPedigreeFunction     0.825
Age                         56.000
Outcome                      1.000
Name: 542, dtype: float64

In [12]:
# Define the function for prediction with uncertainty
def predict_diabetes_progression(model, data, feature_scaler, target_scaler=None, mc_samples=100):
    """
    Predict diabetes progression and uncertainty using MC Dropout
    
    Args:
        model: Trained BNN model
        data: Input data for prediction (DataFrame or dict)
        feature_scaler: Scaler used for features during training
        target_scaler: Optional scaler for target (if used during training)
        mc_samples: Number of Monte Carlo samples
        
    Returns:
        predicted_value: Predicted progression value
        uncertainty: Standard deviation of predictions
    """
    model.eval()
    predictions = []
    
    # Convert input to DataFrame if it's a dictionary
    if isinstance(data, dict):
        data = pd.DataFrame([data])
    
    # Scale features
    scaled_data = feature_scaler.transform(data.values)
    input_tensor = torch.tensor(scaled_data, dtype=torch.float32)
    
    with torch.no_grad():
        for _ in range(mc_samples):
            pred = model(input_tensor, mc_dropout=True)
            predictions.append(pred.item())
    
    predictions = np.array(predictions)
    predicted_value = predictions.mean()
    uncertainty = predictions.std()
    
    # Inverse transform if target was scaled
    if target_scaler is not None:
        predicted_value = target_scaler.inverse_transform([[predicted_value]])[0][0]
        uncertainty = uncertainty * target_scaler.scale_[0]
    
    return predicted_value, uncertainty

# Example new patient data (using your row structure)
new_patient = {
    'Pregnancies': 10,
    'Glucose': 90,
    'BloodPressure': 85,
    'SkinThickness': 32,
    'Insulin': 0,
    'BMI': 34.9,
    'DiabetesPedigreeFunction': 0.825,
    'Age': 56
}

# Convert to DataFrame
patient_df = pd.DataFrame([new_patient])

# Make prediction
predicted_progression, uncertainty = predict_diabetes_progression(
    model=model,
    data=patient_df,
    feature_scaler=feature_scaler,  # Your feature scaler from training
    target_scaler=target_scaler,    # Your target scaler (if used)
    mc_samples=50
)

# Print results
print(f"\nPredicted Diabetes Progression: {predicted_progression:.1f}")
print(f"Uncertainty: ±{uncertainty:.1f}")
print("Interpretation:")
print("- Higher values indicate faster disease progression")
print(f"- 95% confidence interval: [{predicted_progression-2*uncertainty:.1f}, {predicted_progression+2*uncertainty:.1f}]")


Predicted Diabetes Progression: 0.7
Uncertainty: ±0.5
Interpretation:
- Higher values indicate faster disease progression
- 95% confidence interval: [-0.3, 1.6]
